# Stackleberg equilibrium

This notebook is used to find the stackleberg equilibrium for a game in extensive form translated into sequence-form.

In [114]:
from amplpy import AMPL
import pandas as pd

In [115]:
def data_preprocessing(players, sequences, nodes, F, f, utilities):
    sequences_encoded = {player: [i for i in range(len(sequences[player]))] for player in players}
    nodes_encoded = {player: [i for i in range(len(nodes[player]))] for player in players}
    
    utilities_df = {player: pd.DataFrame(utilities[player], index=sequences_encoded['leader'], columns=sequences_encoded['follower']) for player in players}
    F_df = {player: pd.DataFrame(F[player], index=nodes_encoded[player], columns=sequences_encoded[player]) for player in players}
    f_df = {player: pd.DataFrame(f[player], index=nodes_encoded[player]) for player in players}
    M = {player: max(utilities_df[player]) - min(utilities_df[player])  for player in players}
    
    return utilities_df, sequences_encoded, nodes_encoded, F_df, f_df, M

In [116]:
# players, Q, H, F, f, U
def stackleberg_equilibrium(players, sequences, nodes, F, f, utilities):
    utilities_df, sequences_encoded, nodes_encoded, F_df, f_df, M = data_preprocessing(players, sequences, nodes, F, f, utilities)
    
    ampl = AMPL()
    ampl.read('sequence_stackleberg.mod')
    
    ampl.set['Ql'] = sequences_encoded['leader']
    ampl.set['Qf'] = sequences_encoded['follower']
    
    ampl.set['Hl'] = nodes_encoded['leader']
    ampl.set['Hf'] = nodes_encoded['follower']
    
    ampl.param['Fl'] = F_df['leader']
    ampl.param['Ff'] = F_df['follower']
    
    ampl.param['fl'] = f_df['leader']
    ampl.param['ff'] = f_df['follower']
    
    ampl.param['Ml'] = M['leader']
    ampl.param['Mf'] = M['follower']
    
    ampl.param['Ul'] = utilities_df['leader']
    ampl.param['Uf'] = utilities_df['follower']

    ampl.solve(solver='gurobi')
    assert ampl.solve_result == "solved"
    
    obj = ampl.getObjective('obj').value()
    rl = ampl.getVariable('rl').getValues().toPandas()
    rf = ampl.getVariable('rf').getValues().toPandas()

    ampl.close()
    
    return obj, rl, rf

In [117]:
# Game
players = ['leader', 'follower']

# Nodes
H_leader = ['h0', '1.1', '1.2', '1.3']
H_follower = ['h0', '2.1']

H = {'leader': H_leader, 'follower': H_follower}

# Sequences
Q_leader = ['q0', 'L1', 'R1', 'R1L2', 'R1R2', 'R1L3', 'R1R3']
Q_follower = ['q0', 'l1', 'r1']

Q = {'leader': Q_leader, 'follower': Q_follower}

# F
F_leader = [[1,0,0,0,0,0,0],
            [-1,1,1,0,0,0,0],
            [0,0,-1,1,1,0,0],
            [0,0,-1,0,0,1,1]]

F_follower = [[1,0,0],
            [-1,1,1]]

F = {'leader': F_leader, 'follower': F_follower}

# f
f_leader = [1, 0, 0, 0]

f_follower = [1, 0]

f = {'leader': f_leader, 'follower': f_follower}

# utilities
U_leader = [[0, 0, 0],
            [4, 0, 0],
            [0, 0, 0],
            [0, 6, 0],
            [0, 9, 0],
            [0, 0, 2],
            [0, 0, 4]]

U_follower = [[0, 0, 0],
            [4, 0, 0],
            [0, 0, 0],
            [0, 1, 0],
            [0, 3, 0],
            [0, 0, 1],
            [0, 0, 2]]

U = {'leader': U_leader, 'follower': U_follower}

obj, rl, rf = stackleberg_equilibrium(players, Q, H, F, f, U)

print("\nRESULTS:")
print("Optimal strategy for follower:")
for index, qf in rf.iterrows():
    print(Q['follower'][index], "->", qf['rf.val'])
print("Optimal strategy for leader: ")
for index, ql in rl.iterrows():
    print(Q['leader'][index], "->", ql['rl.val'])
print("Optimal objective value: ", obj)

utilities
{'leader':    0  1  2
0  0  0  0
1  4  0  0
2  0  0  0
3  0  6  0
4  0  9  0
5  0  0  2
6  0  0  4, 'follower':    0  1  2
0  0  0  0
1  4  0  0
2  0  0  0
3  0  1  0
4  0  3  0
5  0  0  1
6  0  0  2}
sequences_encoded
{'leader': [0, 1, 2, 3, 4, 5, 6], 'follower': [0, 1, 2]}
nodes_encoded
{'leader': [0, 1, 2, 3], 'follower': [0, 1]}
F_df
{'leader':    0  1  2  3  4  5  6
0  1  0  0  0  0  0  0
1 -1  1  1  0  0  0  0
2  0  0 -1  1  1  0  0
3  0  0 -1  0  0  1  1, 'follower':    0  1  2
0  1  0  0
1 -1  1  1}
f_df
{'leader':    0
0  1
1  0
2  0
3  0, 'follower':    0
0  1
1  0}
M
{'leader': 2, 'follower': 2}


NameError: name 'sys' is not defined